# GitHub Marketplace ಮೂಲಕ Phi-4 Reasoning ಬಳಸಿ ಮಾದರಿ ಇನ್ಫರೆನ್ಸ್

ಈ ನೋಟ್ಬುಕ್ GitHub Marketplace ಮಾದರಿಗಳನ್ನು ಇನ್ಫರೆನ್ಸ್‌ಗಾಗಿ ಹೇಗೆ ಬಳಸಬೇಕೆಂದು ತೋರಿಸುತ್ತದೆ, ವಿಶೇಷವಾಗಿ Microsoft ನ Phi-4-reasoning ಮಾದರಿಯನ್ನು ಬಳಸಿ.

## ಸೆಟ್‌ಅಪ್ ಮತ್ತು ಕಾನ್ಫಿಗರೇಷನ್

ಮೊದಲು, ನಾವು ನಮ್ಮ ವಾತಾವರಣವನ್ನು ಸಂರಚಿಸಿ ಅಗತ್ಯವಿರುವ ಡಿಪೆಂಡೆನ್ಸಿಗಳನ್ನು ಸ್ಥಾಪಿಸುತ್ತೇವೆ.


In [ ]:
# Install required packages
!pip install requests python-dotenv
!pip install azure-ai-inference

### local.env ಫೈಲ್ ಅನ್ನು ಸೆಟ್ ಮಾಡುವುದು

ಈ ನೋಟ್‌ಬುಕ್ ಅನ್ನು ರನ್ ಮಾಡುವ ಮೊದಲು, ಈ ನೋಟ್‌ಬುಕ್ ಇರುವ ಅದೇ ಡೈರೆಕ್ಟರಿಯಲ್ಲಿ ಕೆಳಗಿನ ಚರಗಳೊಂದಿಗೆ `local.env` ಫೈಲ್ ಅನ್ನು ರಚಿಸುವ ಅಗತ್ಯವಿದೆ:

```
# GitHub Configuration
GITHUB_TOKEN=your_personal_access_token_here
GITHUB_INFERENCE_ENDPOINT=https://models.github.ai/inference
GITHUB_MODEL=microsoft/Phi-4-reasoning

# Azure OpenAI Configuration
AZURE_API_KEY=your_azure_api_key_here
AZURE_OPENAI_ENDPOINT=your_azure_endpoint_here
AZURE_OPENAI_MODEL=Phi-4-reasoning
```

**ಸೂಚನೆಗಳು:**

1. ಈ ನೋಟ್‌ಬುಕ್ ಇರುವ ಅದೇ ಫೋಲ್ಡರ್‌ನಲ್ಲಿ `local.env` ಎಂಬ ಹೊಸ ಫೈಲ್ ರಚಿಸಿ
2. ಮೇಲೇ ತೋರಿಸಿರುವ ಮೂರು ಪರಿಸರ ಚರಗಳನ್ನು ಸೇರಿಸಿ
3. `your_personal_access_token_here` ಅನ್ನು ನಿಮ್ಮ GitHub ವೈಯಕ್ತಿಕ ಪ್ರವೇಶ ಟೋಕನ್‌ನಿಂದ ಬದಲಾಯಿಸಿ
4. ಚಿಕ್ಕ ಮಾದರಿಗಾಗಿ, ಐಚ್ಛಿಕವಾಗಿ ನೀವು ಮಾದರಿಯನ್ನು `microsoft/Phi-4-mini-reasoning` ಗೆ ಬದಲಾಯಿಸಬಹುದು

**ಗಮನಿಸಿ:** GitHub ಟೋಕನ್‌ಗೆ AI ಮಾದರಿಗಳ ಸೇವೆಯನ್ನು ಪ್ರವೇಶಿಸಲು ಸರಿಯಾದ ಅನುಮತಿಗಳು ಅಗತ್ಯವಿರುತ್ತವೆ.


## ವಾತಾವರಣ ಚರಗಳನ್ನು ಲೋಡ್ ಮಾಡಿ

ನಾವು ನಮ್ಮ ವಾತಾವರಣ ಚರಗಳನ್ನು `local.env` ಫೈಲ್‌ನಿಂದ ಲೋಡ್ ಮಾಡುತ್ತೇವೆ, ಅದು ನಮ್ಮ GitHub ಟೋಕನ್ ಮತ್ತು ಮಾದರಿ ಮಾಹಿತಿಯನ್ನು ಹೊಂದಿದೆ.


In [ ]:
import os
import requests
import json
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
from dotenv import load_dotenv

# Load variables from local.env file
load_dotenv('local.env')

# Access the environment variables - using values from local.env file
endpoint = os.getenv("GITHUB_INFERENCE_ENDPOINT")
model = os.getenv("GITHUB_MODEL")
token = os.getenv("GITHUB_TOKEN")
azuretoken = os.getenv("AZURE_KEY")
azureendpoint = os.getenv("AZURE_ENDPOINT")
azuremodel = os.getenv("AZURE_MODEL")
# Use fallback values if not found in local.env
if not endpoint:
    endpoint = "https://models.github.ai/inference"
    print("Warning: GITHUB_INFERENCE_ENDPOINT not found in local.env, using default value")

if not model:
    model = "microsoft/Phi-4-reasoning"
    print("Warning: GITHUB_MODEL not found in local.env, using default value")
    print("To change the model to Phi-4-mini-reasoning use \"microsoft/Phi-4-mini-reasoning\"")

if not token:
    raise ValueError("GITHUB_TOKEN not found in local.env file. Please add your GitHub token.")

print(f"Endpoint: {endpoint}")
print(f"Model: {model}")
print(f"azure_ai_image_generation_new.ipynb: {azureendpoint}")
print(f"azuremodel: {azuremodel}")
print(f"azuretoken available: {'Yes' if azuretoken else 'No'}")
print(f"Token available: {'Yes' if token else 'No'}")

## ಮಾದರಿ ಊಹೆಗಾಗಿ ಸಹಾಯಕ ಫಂಕ್ಷನ್‌ಗಳು

GitHub inference API ಜೊತೆಗೆ ಸಂವಹನ ಮಾಡಲು ಸಹಾಯಕ ಫಂಕ್ಷನ್‌ಗಳನ್ನು ರಚಿಸೋಣ.


In [5]:
def generate_completion(prompt, model_id=model, temperature=0.7, max_tokens=10000):
    """Generate a completion using the GitHub inference API"""
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }
    
    payload = {
        "model": model_id,
        "prompt": prompt,
        "temperature": temperature,
        "max_tokens": max_tokens
    }
    
    try:
        # GitHub Models API requires a different endpoint structure
        api_url = f"{endpoint}/v1/chat/completions"
        print(f"Calling API at: {api_url}")
        
        # Modify payload for chat completions format
        chat_payload = {
            "model": model_id,
            "messages": [{"role": "user", "content": prompt}],
            "temperature": temperature,
            "max_tokens": max_tokens
        }
        
        response = requests.post(api_url, headers=headers, json=chat_payload)
        response.raise_for_status()  # Raise exception for 4XX/5XX errors
        result = response.json()
        
        if 'choices' in result and len(result['choices']) > 0:
            # Handle chat completions response format
            if 'message' in result['choices'][0] and 'content' in result['choices'][0]['message']:
                return result['choices'][0]['message']['content']
            # Fall back to the text field if available
            elif 'text' in result['choices'][0]:
                return result['choices'][0]['text']
            else:
                return f"Error: Could not extract content from response: {result['choices'][0]}"
        else:
            return f"Error: Unexpected response format: {result}"
    except Exception as e:
        print(f"Full error details: {str(e)}")
        return f"Error during API call: {str(e)}"

def format_conversation(messages):
    """Format a conversation for the model"""
    # For chat completion API, we'll just return the messages directly
    return messages

def generate_chat_completion(messages, model_id=model, temperature=0.7, max_tokens=1000):
    """Generate a completion using GitHub's chat completions API"""
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }
    
    payload = {
        "model": model_id,
        "messages": messages,
        "temperature": temperature,
        "max_tokens": max_tokens
    }
    
    try:
        api_url = f"{endpoint}/v1/chat/completions"
        print(f"Calling API at: {api_url}")
        
        response = requests.post(api_url, headers=headers, json=payload)
        response.raise_for_status()
        result = response.json()
        
        if 'choices' in result and len(result['choices']) > 0:
            if 'message' in result['choices'][0]:
                return result['choices'][0]['message']['content']
            else:
                return f"Error: Unexpected response format: {result['choices'][0]}"
        else:
            return f"Error: Unexpected response format: {result}"
    except Exception as e:
        print(f"Full error details: {str(e)}")
        return f"Error during API call: {str(e)}"
        
# For backward compatibility with existing code
def format_prompt_legacy(messages):
    """Format a conversation in text format (legacy method)"""
    formatted_prompt = ""
    
    for msg in messages:
        role = msg.get("role", "")
        content = msg.get("content", "")
        
        if role == "user":
            formatted_prompt += f"User: {content}\n\n"
        elif role == "assistant":
            formatted_prompt += f"Assistant: {content}\n\n"
        elif role == "system":
            formatted_prompt += f"{content}\n\n"
    
    formatted_prompt += "Assistant: "
    return formatted_prompt

## ಉದಾಹರಣೆ 1: 9 r's ಗಾಗಿ ಎಷ್ಟು ಸ್ಟ್ರಾಬೆರಿಗಳು?

ನಮ್ಮ ಮೊದಲ ಊಹಾ ಉದಾಹರಣೆಯನ್ನು ನಡಿಸೋಣ, ಇದು ಸ್ಟ್ರಾಬೆರಿಗಳು ಮತ್ತು r's ಬಗ್ಗೆ ಕೇಳುತ್ತದೆ.


In [6]:
example1_messages = [
    {"role": "system", "content": "You are a helpful AI assistant that answers questions accurately and concisely."},
    {"role": "user", "content": "How many strawberries do I need to collect 9 r's?"}
]

print("Messages:")
for msg in example1_messages:
    print(f"{msg['role']}: {msg['content']}")
print("\nGenerating response...\n")

# Use the new chat completion function directly with the messages
response1 = generate_chat_completion(example1_messages)
print("Response:")
print(response1)

Messages:
system: You are a helpful AI assistant that answers questions accurately and concisely.
user: How many strawberries do I need to collect 9 r's?

Generating response...

Calling API at: https://models.github.ai/inference/v1/chat/completions
Response:
<think>User says: "How many strawberries do I need to collect 9 r's?" This appears to be some riddle. Possibly the phrase "9 r's" might be a pun or reference to "r" letters. For example, "How many strawberries do I need to collect 9 r's?" It might be a pun on the phrase "strawberry, nine, r's" or "I need 9 r's" might be a riddle. Alternatively, perhaps the question "How many strawberries do I need to collect 9 r's?" might be a riddle where the answer is something like "9 strawberries" are needed if "strawberry" contains one "r" or two? Let's think: "Strawberry" letter count: S T R A W B E R R Y. Counting letter "r": "strawberry" has two "r"s, one in "straw" and one in "berry" but wait, let's check: "strawberry" letters: S, T, R, A

### ಉದಾಹರಣೆ 1ನ ವಿಶ್ಲೇಷಣೆ

ಈ ಉದಾಹರಣೆಯಲ್ಲಿ, ಮಾದರಿಯು "strawberry" ಎಂಬ ಪದದಲ್ಲಿ ಎರಡು 'r' ಅಕ್ಷರಗಳಿರುವುದನ್ನು ಅರ್ಥಮಾಡಿಕೊಳ್ಳಬೇಕು. ಆದ್ದರಿಂದ, 9 'r'ಗಳನ್ನು ಸಂಗ್ರಹಿಸಲು ನಿಮಗೆ 5 ಸ್ಟ್ರಾಬೆರಿಗಳು ಬೇಕಾಗುತ್ತದೆ (ಒಟ್ಟು 10 'r'ಗಳು), ಅಥವಾ ನಿಖರವಾಗಿ 9 'r'ಗಳನ್ನು ಪಡೆಯಲು 4.5 ಸ್ಟ್ರಾಬೆರಿಗಳು ಬೇಕಾಗುತ್ತವೆ.

ಈ ಸಮಸ್ಯೆಯನ್ನು Phi-4-reasoning model ಹೇಗೆ ನಿರ್ವಹಿಸುತ್ತದೆ ಎಂಬುದನ್ನು ನೋಡೋಣ.


## ಉದಾಹರಣೆ 2: ಒಂದು ಊಹೆಪ್ರಶ್ನೆ ಪರಿಹರಿಸುವುದು

ಈಗ ನಾವು ಇನ್ನೂ ಹೆಚ್ಚು ಸಂಕೀರ್ಣವಾದ ಒಂದು ಉದಾಹರಣೆಯನ್ನು ಪ್ರಯತ್ನಿಸೋಣ - ಬಹು ಉದಾಹರಣೆಗಳಿರುವ ಮಾದರಿ ಗುರುತಿಸುವ ಊಹೆಪ್ರಶ್ನೆ.


In [7]:
example2_messages = [
    {"role": "system", "content": "You are a helpful AI assistant that solves riddles and finds patterns in sequences."},
    {"role": "user", "content": "I will give you a riddle to solve with a few examples, and something to complete at the end"},
    {"role": "user", "content": "nuno Δημήτρης evif Issis 4"},
    {"role": "user", "content": "ntres Inez neves Margot 4"},
    {"role": "user", "content": "ndrei Jordan evlewt Μαρία 9"},
    {"role": "user", "content": "nπέντε Kang-Yuk xis-ytnewt Nubia 21"},
    {"role": "user", "content": "nπέντε Κώστας eerht-ytnewt Μανώλης 18"}, 
    {"role": "user", "content": "nminus one-point-two Satya eno Bill X."},
    {"role": "user", "content": "What is a likely completion for X that is consistent with examples above?"}
]

print("Messages:")
for msg in example2_messages:
    print(f"{msg['role']}: {msg['content'][:50]}...")
print("\nGenerating response...\n")

response2 = generate_chat_completion(example2_messages, temperature=0.2, max_tokens=10000)
print("Response:")
print(response2)

Messages:
system: You are a helpful AI assistant that solves riddles...
user: I will give you a riddle to solve with a few examp...
user: nuno Δημήτρης evif Issis 4...
user: ntres Inez neves Margot 4...
user: ndrei Jordan evlewt Μαρία 9...
user: nπέντε Kang-Yuk xis-ytnewt Nubia 21...
user: nπέντε Κώστας eerht-ytnewt Μανώλης 18...
user: nminus one-point-two Satya eno Bill X....
user: What is a likely completion for X that is consiste...

Generating response...

Calling API at: https://models.github.ai/inference/v1/chat/completions
Response:
<think>We are given a riddle with examples. The riddle is: "I will give you a riddle to solve with a few examples, and something to complete at the end". The examples are:

1. "nuno Δημήτρης evif Issis 4"
2. "ntres Inez neves Margot 4"
3. "ndrei Jordan evlewt Μαρία 9"
4. "nπέντε Kang-Yuk xis-ytnewt Nubia 21"
5. "nπέντε Κώστας eerht-ytnewt Μανώλης 18"
6. "nminus one-point-two Satya eno Bill X."

We are asked: "What is a likely completion for X that is

### ಉದಾಹರಣೆ 2 ರ ವಿಶ್ಲೇಷಣೆ

ಈ ರಹಸ್ಯಪ್ರಶ್ನೆಗೆ ಹಲವಾರು ಭಾಷೆಗಳು ಮತ್ತು ಸಂಖ್ಯಾ ಪ್ರತಿನಿಧಿಗಳಲ್ಲಿ ಇರುವ ಜಟಿಲ ಮಾದರಿಗಳನ್ನು ಗುರುತಿಸುವ ಅಗತ್ಯವಿದೆ. ಮಾರುಕಟ್ಟೆಯಲ್ಲಿ ಮಾದರಿಯನ್ನು what the model needs to understand ಅನ್ನು ವಿಶ್ಲೇಷಿಸೋಣ:

1. “evif” (five) ಮುಂತಾದ ಪದಗಳಲ್ಲಿ ಪದಗಳನ್ನು ಹಿಂದಿನಿಂದ ಬರೆಯಲಾಗಿದೆ ಎಂಬುದನ್ನು ಗುರುತಿಸಿ
2. ವಿಭಿನ್ನ ಭಾಷೆಗಳಲ್ಲಿ ಸಂಖ್ಯೆಗಳ ಗುರುತನ್ನು ಮಾಡಿ (ಉದಾ., ಸ್ಪ್ಯಾನಿಷ್‌ನಲ್ಲಿ "uno", ಗ್ರೀಕ್‌ನಲ್ಲಿ "πέντε")
3. ಸಂಖ್ಯೆಗಳ ಮತ್ತು ಕೊನೆಯ ಅಂಕದ ನಡುವಿನ ಸಂಬಂಧವನ್ನು ಕಂಡುಹಿಡಿಯಿರಿ

ಮಾದರಿ ವಿವಿಧ ಭಾಷೆಗಳಲ್ಲಿ ಮತ್ತು ಸ್ವರೂಪಗಳಲ್ಲಿ ಪ್ರತಿನಿಧಿಸಲಾದ ಮೌಲ್ಯಗಳ ನಡುವಿನ ಗಣಿತೀಯ ಕ್ರಿಯೆಗಳನ್ನೊಳಗೊಂಡಿರಬಹುದು ಎಂದು ತೋರುತ್ತದೆ.


## ವಿಭಿನ್ನ ಪ್ಯಾರಾಮೀಟರ್‌ಗಳೊಂದಿಗೆ ಪ್ರಯೋಗ

ಮತ್ತೆ ಎರಡನೆಯ ಉದಾಹರಣೆಯನ್ನು ವಿಭಿನ್ನ ಟೆಂಪರೇಚರ್ ಸೆಟ್ಟಿಂಗ್‌ಗಳೊಂದಿಗೆ ಪ್ರಯತ್ನಿಸಿ ನೋಡೋಣ, ಇದು ಮಾದರಿಯ ಪ್ರತಿಕ್ರಿಯೆ ಮೇಲೆ ಹೇಗೆ ಪರಿಣಾಮ ಬೀರುತ್ತದೆ ಎಂದು ನೋಡಲು.


In [ ]:
# Try with a higher temperature for more creative responses
response_creative = generate_chat_completion(example2_messages, temperature=0.9, max_tokens=20000)
print("Response with higher temperature (0.9):")
print(response_creative)

## உங்கள் சொந்த உதாரணத்தை உருவாக்குங்கள்

ನೀವು ಮಾದರಿಯ ತರ್ಕ ಸಾಮರ್ಥ್ಯಗಳನ್ನು ಪರೀಕ್ಷಿಸಲು ನಿಮ್ಮದೇ ಉದಾಹರಣೆಗಳನ್ನು ರಚಿಸಬಹುದು. ಪ್ರಾಂಪ್ಟ್‌ಗಳನ್ನು ತಿದ್ದುಪಡಿಸುವುದು ಅಥವಾ ಕೆಳಗೆ ಸಂಪೂರ್ಣವಾಗಿ ಹೊಸ ಪರಿಸ್ಥಿತಿಗಳನ್ನು ರಚಿಸುವುದು ಪ್ರಯತ್ನಿಸಿ.


In [ ]:
# Define your custom prompt here
custom_messages = [
    {"role": "system", "content": "You are a helpful AI assistant that can solve complex problems."},
    {"role": "user", "content": "Your custom prompt here"}
]

# Uncomment the lines below to run your custom prompt
# custom_response = generate_chat_completion(custom_messages)
# print("Response to custom prompt:")
# print(custom_response)

## ಸಾರಾಂಶ

ಈ ನೋಟ್‌ಬುಕ್ GitHub Marketplace ಮಾದರಿಗಳನ್ನು ಇನ್ಫರೆನ್ಸ್‌ಗಾಗಿ ಹೇಗೆ ಬಳಸುವುದು ಎಂದು ತೋರಿಸಿತು, ವಿಶೇಷವಾಗಿ ತಾರ್ಕಿಕ ಸಮಸ್ಯೆಗಳು ಮತ್ತು ರಿಡ್ಲ್ಸ್‌ಗಳನ್ನು ಪರಿಹರಿಸಲು Phi-4-reasoning ಮಾದರಿಯನ್ನು ಬಳಸುವುದು.

ಪ್ರಮುಖಾಂಶಗಳು:
1. GitHub ಟೋಕನ್‌ಗಳೊಂದಿಗೆ ಪ್ರಮಾಣೀಕರಣವನ್ನು ಸ್ಥಾಪಿಸುವುದು
2. ಉತ್ತಮ ಇನ್ಫರೆನ್ಸ್‌ಗಾಗಿ ಪ್ರಾಂಪ್ಟ್‌ಗಳನ್ನು ಸ್ವರೂಪಗೊಳಿಸುವುದು
3. ಪ್ರತಿಕ್ರಿಯೆಯ ವೈವಿಧ್ಯತೆಯನ್ನು ನಿಯಂತ್ರಿಸಲು temperature 같은 ಮಾದರಿ ಪರಿಮಾಣಗಳನ್ನು ನಿರ್ವಹಿಸುವುದು
4. ವಿವಿಧ ರೀತಿಯ ಸಮಸ್ಯೆಗಳ ಮೇಲೆ ಮಾದರಿಯ ತರ್ಕ ಸಾಮರ್ಥ್ಯವನ್ನು ಪರೀಕ್ಷಿಸುವುದು

ನಿಮ್ಮ GitHub ಟೋಕನ್ ಅನ್ನು ಸುರಕ್ಷಿತವಾಗಿರಿಸಿ ಮತ್ತು ಅದನ್ನು ಸಾರ್ವಜನಿಕ ರೆಪೊಸಿಟರಿ ಅಥವಾ ನೋಟ್‌ಬುಕ್‌ಗಳಲ್ಲಿ ಹಂಚಿಕೊಳ್ಳಬೇಡಿ.


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
ನಿರಾಕರಣೆ:
ಈ ದಸ್ತಾವೇಜನ್ನು AI ಅನುವಾದ ಸೇವೆ Co‑op Translator (https://github.com/Azure/co-op-translator) ಬಳಸಿ ಅನುವಾದಿಸಲಾಗಿದೆ. ನಾವು ನಿಖರತೆಯತ್ತ ಪ್ರಯತ್ನಿಸಿದರೂ, ದಯವಿಟ್ಟು ಗಮನಿಸಿ — ಸ್ವಯಂಚಾಲಿತ ಅನುವಾದಗಳಲ್ಲಿ ತಪ್ಪುಗಳು ಅಥವಾ ಅನಿಖರತೆಗಳು ಇರಬಹುದು. ಮೂಲ ಭಾಷೆಯಲ್ಲಿ ಇರುವ ಮೂಲ ದಸ್ತಾವೇಜನ್ನು ಅಧಿಕೃತ ಮತ್ತು ನಂಬಬಹುದಾದ ಮೂಲವೆಂದು ಪರಿಗಣಿಸಬೇಕು. ಗಂಭೀರ ಮಾಹಿತಿಗಾಗಿ, ವೃತ್ತಿಪರ ಮಾನವ ಅನುವಾದವನ್ನು ಶಿಫಾರಸು ಮಾಡಲಾಗುತ್ತದೆ. ಈ ಅನುವಾದವನ್ನು ಬಳಸಿಕೊಂಡು ಉಂಟಾಗುವ ಯಾವುದೇ ಗೊಂದಲಗಳು ಅಥವಾ ತಪ್ಪಾಗಿ ವ್ಯಾಖ್ಯಾನಿಸುವಿಕೆಗಳಿಗಾಗಿ ನಾವು ಜವಾಬ್ದಾರಿಯಾಗುವುದಿಲ್ಲ.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
